<a href="https://colab.research.google.com/github/abdulrahimzulfiqar/English-to-Spanish-Translation-Using-LSTM-Neural-Machine-Translation-/blob/main/An_Encoder%E2%80%93Decoder_Network_for_Neural_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import zipfile
from pathlib import Path

# Download manually to a known folder (not using TensorFlow's default)
url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
zip_path = tf.keras.utils.get_file(
    fname="spa-eng.zip",
    origin=url,
    extract=False,        # ⛔ Don't extract automatically
    cache_dir="/content", # ✅ Use /content for Colab visibility
    cache_subdir=""
)

# ✅ Extract manually to a visible location
extract_path = Path("/content/spa-eng-data")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# ✅ Read the file
text_path = extract_path / "spa-eng" / "spa.txt"
text = text_path.read_text(encoding='utf-8')

# ✅ Show preview
print(text[:500])


2638744/2638744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Go.	Ve.
Go.	Vete.
Go.	Vaya.
Go.	Váyase.
Hi.	Hola.
Run!	¡Corre!
Run.	Corred.
Who?	¿Quién?
Fire!	¡Fuego!
Fire!	¡Incendio!
Fire!	¡Disparad!
Help!	¡Ayuda!
Help!	¡Socorro! ¡Auxilio!
Help!	¡Auxilio!
Jump!	¡Salta!
Jump.	Salte.
Stop!	¡Parad!
Stop!	¡Para!
Stop!	¡Pare!
Wait!	¡Espera!
Wait.	Esperen.
Go on.	Continúa.
Go on.	Continúe.
Hello!	Hola.
I ran.	Corrí.
I ran.	Corría.
I try.	Lo intento.
I won!	¡He ganado!
Oh no!	¡Oh, no!
Relax.	Tomátelo con soda.
Smile.	Sonríe.
Attack!	¡Al ataque!
Attack!	¡Atacad!
Ge


In [ ]:
import numpy as np
text = text.replace("¡", "").replace("¿", "")
pairs = [line.split("\t") for line in text.splitlines()]
np.random.shuffle(pairs)
sentences_en, sentences_es = zip(*pairs) # separates the pairs into 2 lists

In [ ]:
for i in range(3):
  print(sentences_en[i], "=>", sentences_es[i])

It's raining cats and dogs. => Está lloviendo a cántaros.
Tom left his briefcase on his desk. => Tom dejó la maleta sobre su escritorio.
I went to China two years ago. => Fui a China hace dos años.


In [ ]:
vocab_size = 1000
max_length = 50
text_vec_layer_en = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_es = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f"startofseq {s} endofseq" for s in sentences_es])

In [ ]:
text_vec_layer_en.get_vocabulary()[:10],text_vec_layer_es.get_vocabulary()[:10]

(['',
  '[UNK]',
  np.str_('the'),
  np.str_('i'),
  np.str_('to'),
  np.str_('you'),
  np.str_('tom'),
  np.str_('a'),
  np.str_('is'),
  np.str_('he')],
 ['',
  '[UNK]',
  np.str_('startofseq'),
  np.str_('endofseq'),
  np.str_('de'),
  np.str_('que'),
  np.str_('a'),
  np.str_('no'),
  np.str_('tom'),
  np.str_('la')])

In [ ]:
X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_en[100_000:])
X_train_dec = tf.constant([f"startofseq {s}" for s in sentences_es[:100_000]])
X_valid_dec = tf.constant([f"startofseq {s}" for s in sentences_es[100_000:]])
Y_train = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[:100_000]])
Y_valid = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[100_000:]])

In [ ]:
encoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)

In [ ]:
embed_size = 128
encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

In [ ]:
encoder = tf.keras.layers.LSTM(512, return_state=True)
encoder_outputs, *encoder_state = encoder(encoder_embeddings)

In [ ]:
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

In [ ]:
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)

In [ ]:
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 79s 24ms/step - accuracy: 0.0510 - loss: 3.5182 - val_accuracy: 0.0746 - val_loss: 2.1673
Epoch 2/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 75s 22ms/step - accuracy: 0.0787 - loss: 1.9551 - val_accuracy: 0.0870 - val_loss: 1.6514
Epoch 3/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 82s 22ms/step - accuracy: 0.0913 - loss: 1.4539 - val_accuracy: 0.0931 - val_loss: 1.4286
Epoch 4/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 69s 22ms/step - accuracy: 0.0990 - loss: 1.1887 - val_accuracy: 0.0957 - val_loss: 1.3330
Epoch 5/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 87s 24ms/step - accuracy: 0.1042 - loss: 1.0026 - val_accuracy: 0.0966 - val_loss: 1.2991
Epoch 6/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 77s 22ms/step - accuracy: 0.1088 - loss: 0.8614 - val_accuracy: 0.0971 - val_loss: 1.2946
Epoch 7/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 82s 22ms/step - accuracy: 0.1125 - loss: 0.7431 - val_accuracy: 0.0975 - val_loss: 1.3065
Epoch 8/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 82s 22ms/step - accuracy: 0.1162 -

In [ ]:
def translate(sentence_en):
    translation = ""
    for word_idx in range(max_length):
        X = tf.constant([sentence_en])  # raw English sentence string
        X_dec = tf.constant(["startofseq " + translation])  # raw partial Spanish translation

        y_proba = model.predict((X, X_dec))[0, word_idx]
        predicted_word_id = np.argmax(y_proba)
        predicted_word = text_vec_layer_es.get_vocabulary()[predicted_word_id]

        if predicted_word == "endofseq":
            break

        translation += " " + predicted_word
    return translation.strip()


In [ ]:
translate('i like car')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


'me gusta el carro'

In [ ]:
translate('I like soccer')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


'me gusta el fútbol'